In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

<h4>Getting links to articles about Russia published in the last month</h4>

In [2]:
def get_links(html):
    r = requests.get(html)
    soup = bs(r.content)
    # Список, куда будут помещены ссылки
    links_list = []
    # Блоки новостей
    news_blocks = soup.find_all('div', class_='NiLAwe')
    # Текущее дата-время
    current_datetime = datetime.now()
    for news in news_blocks:
        time = news.find('time')
        # Выписываю дату, время и соединяю их, преобразовав в нужный формат
        news_date = time['datetime'][:10]
        news_time = time['datetime'][11:-1]
        news_datetime = datetime.strptime(news_date + ' ' + news_time, "%Y-%m-%d %H:%M:%S")
        # Если статья вышла не позднее месяца назад, то добавляю ссылку в список. Иначе - следующая новость
        if current_datetime - timedelta(days = 31) <= news_datetime:
            news_link = news.find('a')['href']
            correct_link = 'https://news.google.com' + news_link[1:]
            links_list.append(correct_link)
        else:
            continue
    return links_list

In [3]:
# Запуская ф-ию и получаю список ссылок на новости, вышедшие не позднее месяца назад
links_list = get_links("https://news.google.com/search?q=russia&hl=en-US&gl=US&ceid=US%3Aen")
len(links_list)

100

<h4>Getting news content from relevant links</h4>

In [4]:

news_list = []
print('Количество новостей: ', len(links_list), '\n')
count = 0
for link in links_list:
    try:
        print(f'{count} итерация:')
        print('Выполняется request')
        link_request = requests.get(link, timeout = (10, 30))
        print('request выполнен успешно. Делаем суп...')
        link_soup = bs(link_request.content)
        print('Суп сделан!')
    except:
        count += 1
        print(f'Не удалось скачать {count} статью')
        continue
    news_list.append(link_soup.find('title').get_text())
    p = link_soup.find_all(['p', 'h1'])
    content_text = []
    for i in p:
        content_text.append(i.get_text())
    content = '\n'.join(content_text)
    news_list.append(content)
    count += 1
    print(f'{count} статья скачана')

Количество новостей:  100 

0 итерация:
Выполняется request
request выполнен успешно. Делаем суп...
Суп сделан!
1 статья скачана
1 итерация:
Выполняется request
request выполнен успешно. Делаем суп...
Суп сделан!
2 статья скачана
2 итерация:
Выполняется request
request выполнен успешно. Делаем суп...
Суп сделан!
3 статья скачана
3 итерация:
Выполняется request
request выполнен успешно. Делаем суп...
Суп сделан!
4 статья скачана
4 итерация:
Выполняется request
request выполнен успешно. Делаем суп...
Суп сделан!
5 статья скачана
5 итерация:
Выполняется request
request выполнен успешно. Делаем суп...
Суп сделан!
6 статья скачана
6 итерация:
Выполняется request
request выполнен успешно. Делаем суп...
Суп сделан!
7 статья скачана
7 итерация:
Выполняется request
request выполнен успешно. Делаем суп...
Суп сделан!
8 статья скачана
8 итерация:
Выполняется request
request выполнен успешно. Делаем суп...
Суп сделан!
9 статья скачана
9 итерация:
Выполняется request
request выполнен успешно. Делае

In [11]:
#Removing outliers from the list (unsuccessfully loaded articles)
for idx, news in enumerate(news_list):
    if news == 'Page Not Found' or news == '403 Forbidden':
        news_list.pop(idx)    

<h4>Save news list to JSON</h4>

In [13]:
import json
def save_data(title, data):
    with open(title, 'w', encoding = 'utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [14]:
def load_data(title):
    with open(title, encoding = 'utf-8') as f:
        return json.load(f)

In [15]:
save_data('all_text_about_russia.json', news_list)